In [ ]:
# Goal of this notebook is to help with migrating manual Excel reports to automated Hadoop/Tableau platform
# It reads Excel WB and extracts information about fields from pivot tables
# 4 main categories of fields - actively used in pivot table, not in usage, calculated field, "group by' (dimensions) 

In [ ]:
import openpyxl
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import datetime
import re

import multiprocessing
import sys

In [ ]:
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl') # slicer extension warning

In [ ]:
excel_list = os.listdir('excel_pivot_extractor/')

In [ ]:
excel_list

In [ ]:
df_write = pd.DataFrame()

In [ ]:
# create empty excel sheet for appending
df_write.to_excel('excel_pivot_extractor/extracted_pivot_details.xlsx')

In [ ]:
# wb = openpyxl.load_workbook('excel_pivot_extractor/inb_main_INB 2.0 monitor report 20220617_no_data.xlsx')

In [ ]:
# open the excel sheet for writing in openpyxl
writer = pd.ExcelWriter('excel_pivot_extractor/extracted_pivot_details.xlsx', engine='openpyxl', mode='a')

In [ ]:
# list of all excel workbook in the folder
excel_list = [s for s in excel_list if re.search('xlsx', s)]
excel_list.sort()

In [ ]:
# four categories of fields I want to gather
list_of_fields = ['used_field_list','notu_field_list','calc_field_list','group_field_list']

 core function for extracting pivot fields for MULTIPROCESSING
def extr_pivots(num_rep):
    exc_rep = excel_list[num_rep]
    used_field_list = []
    notu_field_list = []
    calc_field_list = []
    group_field_list = []
    # print the file to be read
    print("Loading "+exc_rep+" started at " + str(datetime.datetime.now().strftime("%B %-d %Y %H:%M:%S")))
    
    # the performance bottleneck is loading the workbooks especially with data sheets
    # not to mention that openpyxl doesn't like formatted (coloured) pivot tables
    wb = openpyxl.load_workbook('excel_pivot_extractor/'+exc_rep)
    sheets_len = len(wb._sheets)
    pivots_len = 0
    for sh in range(0,len(wb._sheets)):
        pivots_len = pivots_len + len(wb._sheets[sh]._pivots)
    
    # print the file, execution time and details of how many pivots and sheets were processed
    print(exc_rep + " has succesfully loaded and contains "+str(sheets_len)+" sheets and "+str(pivots_len)+" pivots. Finished at " + str(datetime.datetime.now().strftime("%B %-d %Y %H:%M:%S")))
    
    # sheets in WB
    for sh in range(0,len(wb._sheets)):
        # pivots in a sheet
        for pi in range(0,len(wb._sheets[sh]._pivots)):
            # fields in a pivot
            for ca in range(0,len(wb._sheets[sh]._pivots[pi].cache.cacheFields)):
                # if a property "sharedItems" of the field is existing then it is available in the pivot table  
                try: 
                    if wb._sheets[sh]._pivots[pi].cache.cacheFields[ca].sharedItems.count > 0:
                        used_field_list.append(wb._sheets[sh]._pivots[pi].cache.cacheFields[ca].name)
                    else:
                        notu_field_list.append(wb._sheets[sh]._pivots[pi].cache.cacheFields[ca].name)
                except:
                    # formula for calculated fields
                    try: 
                        if len(wb._sheets[sh]._pivots[pi].cache.cacheFields[ca].formula) > 0:
                            calc_field_list.append(wb._sheets[sh]._pivots[pi].cache.cacheFields[ca].name)
                    except: 
                        # field group for group by dimension
                        try:
                            if type(wb._sheets[sh]._pivots[pi].cache.cacheFields[ca].fieldGroup) == openpyxl.pivot.cache.FieldGroup:
                                group_field_list.append(wb._sheets[sh]._pivots[pi].cache.cacheFields[ca].name)
                        except:
                            print("Didnt assign "+ (wb._sheets[sh]._pivots[pi].cache.cacheFields[ca].name))

    # deduplicating output for each workbook
    used_field_list = pd.DataFrame(np.unique(used_field_list))
    notu_field_list = pd.DataFrame(np.unique(notu_field_list))
    calc_field_list = pd.DataFrame(np.unique(calc_field_list))
    group_field_list = pd.DataFrame(np.unique(group_field_list))

    # output part
    out_dic = {}
    out_dic['Name'] = exc_rep[:6]
    #write each DataFrame to a specific sheet
    for i, ff in enumerate(list_of_fields):
            out_dic[ff] = eval(ff)
    return out_dic

 This function allowed me to write into excel WB simultaneously however each time an instance of the multiprocess finishes it saves the workbook and overwrites the previous instance
def mp_handler():
    pool_obj = multiprocessing.Pool()
    
    # imap_unordered() function iterates items in the iterable one at a time and issue a task in the process pool. 
    # That calls the specified function on the iterable. Then returns an iterable of return values. 
    # The return values are yielded in the order that tasks are completed, not the order that the tasks were issued to the process pool.
    for i, result in enumerate(pool_obj.imap_unordered(extr_pivots, range(0,len(excel_list))),1):
        
        # the four categories of fields
        for c, ff in enumerate(list_of_fields):
            result[ff].to_excel(writer, sheet_name=(result['Name']), index=False, startcol = c)
        # progress bar formatted as stderr instead of stdout to distinguish the progress in the output
        sys.stderr.write('\rdone {0:%}'.format(i/len(excel_list)))

if __name__=='__main__':
    mp_handler()

In [ ]:
# definition of 4 category lists outside of the loop
used_field_list_tot = []
notu_field_list_tot = []
calc_field_list_tot = []
group_field_list_tot = []

In [ ]:
# main part to extract pivot fields from the workbooks 
for exc_rep in tqdm(excel_list):
    #for exc_rep in excel_list:
    used_field_list = []
    notu_field_list = []
    calc_field_list = []
    group_field_list = []
    
    # print the file to be read
    print("Loading "+exc_rep+" WB - started at " + str(datetime.datetime.now().strftime("%B %-d %Y %H:%M:%S")))
    
    # the performance bottleneck is loading the workbooks especially with data sheets
    # not to mention that openpyxl doesn't like formatted (coloured) pivot tables
    wb = openpyxl.load_workbook('excel_pivot_extractor/'+exc_rep)
    sheets_len = len(wb._sheets)
    pivots_len = 0
    for sh in range(0,len(wb._sheets)):
        pivots_len = pivots_len + len(wb._sheets[sh]._pivots)
    
    # print the file, execution time and details of how many pivots and sheets were processed
    print("WB has succesfully loaded and contains "+str(sheets_len)+" sheets and "+str(pivots_len)+" pivots. Finished at " + str(datetime.datetime.now().strftime("%B %-d %Y %H:%M:%S")))
    
    # sheets in WB
    for sh in range(0,len(wb._sheets)):
        # pivots in a sheet
        for pi in range(0,len(wb._sheets[sh]._pivots)):
            # fields in a pivot
            for ca in range(0,len(wb._sheets[sh]._pivots[pi].cache.cacheFields)):
                # if a property "sharedItems" of the field is existing then it is available in the pivot table  
                try: 
                    if wb._sheets[sh]._pivots[pi].cache.cacheFields[ca].sharedItems.count > 0:
                        used_field_list.append(wb._sheets[sh]._pivots[pi].cache.cacheFields[ca].name)
                    else:
                        notu_field_list.append(wb._sheets[sh]._pivots[pi].cache.cacheFields[ca].name)
                except:
                    # formula stands for calculated fields
                    try: 
                        if len(wb._sheets[sh]._pivots[pi].cache.cacheFields[ca].formula) > 0:
                            calc_field_list.append(wb._sheets[sh]._pivots[pi].cache.cacheFields[ca].name)
                    except: 
                        # field group for group by dimension
                        try:
                            if type(wb._sheets[sh]._pivots[pi].cache.cacheFields[ca].fieldGroup) == openpyxl.pivot.cache.FieldGroup:
                                group_field_list.append(wb._sheets[sh]._pivots[pi].cache.cacheFields[ca].name)
                        except:
                            print("Didnt assign "+ (wb._sheets[sh]._pivots[pi].cache.cacheFields[ca].name))

    # deduplicating output for each workbook
    used_field_list = pd.DataFrame(np.unique(used_field_list))
    notu_field_list = pd.DataFrame(np.unique(notu_field_list))
    calc_field_list = pd.DataFrame(np.unique(calc_field_list))
    group_field_list = pd.DataFrame(np.unique(group_field_list))

    used_field_list_tot.append(used_field_list)
    notu_field_list_tot.append(notu_field_list)
    calc_field_list_tot.append(calc_field_list)
    group_field_list_tot.append(group_field_list)
    
    #write each DataFrame to a specific sheet
    for i, ff in enumerate(list_of_fields):
            eval(ff).to_excel(writer, sheet_name=(exc_rep[:6]), index=False, startcol = i)

    #close the Pandas Excel writer and output the Excel file
    writer.save()

In [ ]:
# for each category count appearances and prepare summary, TBD - as a loop of each category 

In [ ]:
flat_list_used_field = pd.DataFrame()
for x in used_field_list_tot:
    try: 
        x[0] = x[0].str.lower()
        flat_list_used_field = flat_list_used_field.append(x,ignore_index=True)
    except: print(x)

flat_list_used_field = flat_list_used_field.value_counts()
flat_list_used_field

In [ ]:
flat_list_notu_used_field = pd.DataFrame()
for x in notu_field_list_tot:
    try: 
        x[0] = x[0].str.lower()
        flat_list_notu_used_field = flat_list_notu_used_field.append(x,ignore_index=True)
    except: print(x)
flat_list_notu_used_field = flat_list_notu_used_field.value_counts()
flat_list_notu_used_field

In [ ]:
flat_list_calc_field = pd.DataFrame()
for x in calc_field_list_tot:
    try: 
        x[0] = x[0].str.lower()
        flat_list_calc_field = flat_list_calc_field.append(x,ignore_index=True)
    except: print(x)
flat_list_calc_field = flat_list_calc_field.value_counts()
flat_list_calc_field

In [ ]:
flat_list_group_field = pd.DataFrame()
for x in group_field_list_tot:
    try: 
        x[0] = x[0].str.lower()
        flat_list_group_field = flat_list_group_field.append(x,ignore_index=True)
    except: print(x)

flat_list_group_field = flat_list_group_field.value_counts()
flat_list_group_field

In [ ]:
# finalize the summary
summary_tot = pd.concat([flat_list_used_field,flat_list_notu_used_field,flat_list_calc_field,flat_list_group_field],axis=1)
summary_tot.rename(columns = {0:'used_fields', 1:'not_used_fields', 2:'calculated_fields', 3:'groupby_fields'}, inplace = True)

In [ ]:
# write it to the excel sheet
summary_tot.to_excel(writer, sheet_name=('Total'), index=True)
writer.save()

In [ ]:
writer.close()